# Coursera Capstone Project Of Segmenting and Clustering Neighborhoods in Toronto

### 1. Web Scraping and Prepare DataFrame

In [57]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [58]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipage = requests.get(url).text
soup = BeautifulSoup(wikipage,'lxml')
table = soup.find('table')

In [59]:
PCodes,boros,neighs = [],[],[]
for row in table.find_all('tr'):
    cols = row.find_all('td')
    for col in cols:
        # Assuming the text data of in the cell will have necessary information as required
        info = col.get_text().strip()
        if 'not assigned' not in info.lower(): # Assuming if the Postal Code has not been assigned, ignore the data
            PCodes.append(info[:3])
            i = info.find('(')
            boros.append(info[3:i])
            neighs.append(info[i+1:-1].replace('/',','))
        
datadict = {'PostalCode':PCodes,'Borough':boros,'Neighbourhood':neighs}
toronto_df = pd.DataFrame()
toronto_df = toronto_df.from_dict(datadict)
toronto_df.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills)Nort
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [60]:
toronto_df.shape

(103, 3)

In [ ]:
### 2. Getting Latitude and longitude of the data

In [61]:
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [62]:
lat_lon.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df3 = pd.merge(toronto_df,lat_lon,on='PostalCode')
df3.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


### Visualizing all the Neighbourhoods of the above data frame using Folium

In [63]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize

import numpy as np

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium

In [69]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df3['Latitude'],df3['Longitude'],df3['Borough'],df3['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

### Using KMeans clustering for the clsutering of the neighbourhoods

In [65]:
k=5
toronto_clustering = df3.drop(['PostalCode','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df3.insert(0, 'Cluster Labels', kmeans.labels_)



In [68]:
#create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df3['Latitude'], df3['Longitude'], df3['Neighbourhood'], df3['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters